In [1]:
import pandas as pd
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
import aequitas.plot as ap 

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('compas_for_aequitas.csv')

In [3]:
print(df.shape)

(7214, 6)


In [4]:
df.head()

,entity_id,score,label_value,race,sex,age_cat
0,1,0.0,0,Other,Male,Greater than 45
1,3,0.0,1,African-American,Male,25 - 45
2,4,0.0,1,African-American,Male,Less than 25
3,5,1.0,0,African-American,Male,Less than 25
4,6,0.0,0,Other,Male,25 - 45


In [6]:
df['race'].value_counts()

race
African-American    3696
Caucasian           2454
Hispanic             637
Other                377
Asian                 32
Native American       18
Name: count, dtype: int64

In [7]:
df = df[~df['race'].isin(['Asian', 'Native American'])]

## Create cross table

In [8]:
group = Group()
xtab, _ = group.get_crosstabs(df)

xtab.head(10)

,model_id,score_threshold,k,attribute_name,attribute_value,accuracy,tpr,tnr,for,fdr,...,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev
0,0,binary 0/1,3297,race,African-American,0.638258,0.720147,0.551532,0.349540,0.370285,...,0.588203,805,532,990,1369,1901,1795,3696,7164,0.514340
1,0,binary 0/1,3297,race,Caucasian,0.669927,0.522774,0.765457,0.288125,0.408665,...,0.348003,349,461,1139,505,966,1488,2454,7164,0.393643
2,0,binary 0/1,3297,race,Hispanic,0.660911,0.443966,0.785185,0.288591,0.457895,...,0.298273,87,129,318,103,232,405,637,7164,0.364207
3,0,binary 0/1,3297,race,Other,0.665782,0.323308,0.852459,0.302013,0.455696,...,0.209549,36,90,208,43,133,244,377,7164,0.352785
4,0,binary 0/1,3297,sex,Female,0.652988,0.607287,0.678212,0.242197,0.489796,...,0.423326,288,194,607,300,494,895,1389,7164,0.355652
5,0,binary 0/1,3297,sex,Male,0.652468,0.628196,0.674350,0.332029,0.365079,...,0.469091,989,1018,2048,1720,2738,3037,5775,7164,0.474113
6,0,binary 0/1,3297,age_cat,25 - 45,0.647073,0.625399,0.665607,0.324885,0.384736,...,0.468528,736,705,1465,1177,1882,2201,4083,7164,0.460936
7,0,binary 0/1,3297,age_cat,Greater than 45,0.702945,0.424797,0.830841,0.241468,0.464103,...,0.249680,181,283,889,209,492,1070,1562,7164,0.314981
8,0,binary 0/1,3297,age_cat,Less than 25,0.615537,0.738928,0.455371,0.426667,0.362173,...,0.654378,360,224,301,634,858,661,1519,7164,0.564845


# Calculate bias

In [10]:
bias = Bias()

bias_df = bias.get_disparity_predefined_groups(
    xtab,
    original_df=df,
    ref_groups_dict={
        'race': 'Caucasian',
        'sex': 'Male',
        'age_cat': '25 - 45'
    },
    alpha=0.05,
    mask_significance=True
)

bias_df.head(10)

,model_id,score_threshold,k,attribute_name,attribute_value,accuracy,tpr,tnr,for,fdr,...,ppr_ref_group_value,pprev_ref_group_value,precision_ref_group_value,fdr_ref_group_value,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value
0,0,binary 0/1,3297,race,African-American,0.638258,0.720147,0.551532,0.349540,0.370285,...,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian
1,0,binary 0/1,3297,race,Caucasian,0.669927,0.522774,0.765457,0.288125,0.408665,...,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian
2,0,binary 0/1,3297,race,Hispanic,0.660911,0.443966,0.785185,0.288591,0.457895,...,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian
3,0,binary 0/1,3297,race,Other,0.665782,0.323308,0.852459,0.302013,0.455696,...,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian
4,0,binary 0/1,3297,sex,Female,0.652988,0.607287,0.678212,0.242197,0.489796,...,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male
5,0,binary 0/1,3297,sex,Male,0.652468,0.628196,0.674350,0.332029,0.365079,...,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male
6,0,binary 0/1,3297,age_cat,25 - 45,0.647073,0.625399,0.665607,0.324885,0.384736,...,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45
7,0,binary 0/1,3297,age_cat,Greater than 45,0.702945,0.424797,0.830841,0.241468,0.464103,...,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45
8,0,binary 0/1,3297,age_cat,Less than 25,0.615537,0.738928,0.455371,0.426667,0.362173,...,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45


In [11]:
bias.get_disparity_major_group(xtab, original_df=df, alpha=0.05, mask_significance=True).head(10)

,model_id,score_threshold,k,attribute_name,attribute_value,accuracy,tpr,tnr,for,fdr,...,ppr_ref_group_value,pprev_ref_group_value,precision_ref_group_value,fdr_ref_group_value,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value
0,0,binary 0/1,3297,race,African-American,0.638258,0.720147,0.551532,0.349540,0.370285,...,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American
1,0,binary 0/1,3297,race,Caucasian,0.669927,0.522774,0.765457,0.288125,0.408665,...,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American
2,0,binary 0/1,3297,race,Hispanic,0.660911,0.443966,0.785185,0.288591,0.457895,...,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American
3,0,binary 0/1,3297,race,Other,0.665782,0.323308,0.852459,0.302013,0.455696,...,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American,African-American
4,0,binary 0/1,3297,sex,Female,0.652988,0.607287,0.678212,0.242197,0.489796,...,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male
5,0,binary 0/1,3297,sex,Male,0.652468,0.628196,0.674350,0.332029,0.365079,...,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male
6,0,binary 0/1,3297,age_cat,25 - 45,0.647073,0.625399,0.665607,0.324885,0.384736,...,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45
7,0,binary 0/1,3297,age_cat,Greater than 45,0.702945,0.424797,0.830841,0.241468,0.464103,...,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45
8,0,binary 0/1,3297,age_cat,Less than 25,0.615537,0.738928,0.455371,0.426667,0.362173,...,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45,25 - 45


In [12]:
fairness = Fairness()
fairness_df = fairness.get_group_value_fairness(bias_df)
fairness_df.head(10)

,model_id,score_threshold,k,attribute_name,attribute_value,accuracy,tpr,tnr,for,fdr,...,FNR Parity,TPR Parity,TNR Parity,NPV Parity,Precision Parity,TypeI Parity,TypeII Parity,Equalized Odds,Unsupervised Fairness,Supervised Fairness
0,0,binary 0/1,3297,race,African-American,0.638258,0.720147,0.551532,0.349540,0.370285,...,False,False,False,True,True,False,False,False,False,False
1,0,binary 0/1,3297,race,Caucasian,0.669927,0.522774,0.765457,0.288125,0.408665,...,True,True,True,True,True,True,True,True,True,True
2,0,binary 0/1,3297,race,Hispanic,0.660911,0.443966,0.785185,0.288591,0.457895,...,True,True,True,True,True,True,True,True,False,True
3,0,binary 0/1,3297,race,Other,0.665782,0.323308,0.852459,0.302013,0.455696,...,False,False,True,True,True,False,False,False,False,False
4,0,binary 0/1,3297,sex,Female,0.652988,0.607287,0.678212,0.242197,0.489796,...,True,True,True,True,True,False,False,True,False,False
5,0,binary 0/1,3297,sex,Male,0.652468,0.628196,0.674350,0.332029,0.365079,...,True,True,True,True,True,True,True,True,True,True
6,0,binary 0/1,3297,age_cat,25 - 45,0.647073,0.625399,0.665607,0.324885,0.384736,...,True,True,True,True,True,True,True,True,True,True
7,0,binary 0/1,3297,age_cat,Greater than 45,0.702945,0.424797,0.830841,0.241468,0.464103,...,False,False,True,True,True,False,False,False,False,False
8,0,binary 0/1,3297,age_cat,Less than 25,0.615537,0.738928,0.455371,0.426667,0.362173,...,False,True,False,True,True,False,False,False,False,False


In [13]:
overall_fairness = fairness.get_overall_fairness(fairness_df)
print(overall_fairness)

{'Unsupervised Fairness': False, 'Supervised Fairness': False, 'Overall Fairness': False}


In [14]:
metrics = ['fpr', 'fnr', 'for']
disparity_tolerance = 1.25

ap.summary(bias_df, metrics, fairness_threshold=disparity_tolerance)

c:\Users\jahaz\.conda\envs\udacity_mlops\lib\site-packages\aequitas\plot\summary_chart.py:373: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  .agg(
c:\Users\jahaz\.conda\envs\udacity_mlops\lib\site-packages\aequitas\plot\summary_chart.py:373: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  .agg(
c:\Users\jahaz\.conda\envs\udacity_mlops\lib\site-packages\aequitas\plot\summary_chart.py:373: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg

alt.VConcatChart(...)